# A Basic Math Agent

This chatbot allows users to ask basic math questions and get answers -- its like an LLM that knows how to use a calculator. This example is very simplistic but it shows you could build more sophisticated agents using Llama Index and visualize them with Gradio.

In [1]:
import os



In [5]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool

def multiply(a: float, b: float) -> float:
    """Multiply two numbers and returns the product"""
    return a * b

multiply_tool = FunctionTool.from_defaults(fn=multiply)

def add(a: float, b: float) -> float:
    """Add two numbers and returns the sum"""
    return a + b

add_tool = FunctionTool.from_defaults(fn=add)

def subtract(a: float, b: float) -> float:
    """Subtract two numbers and returns the difference"""
    return a - b

subtract_tool = FunctionTool.from_defaults(fn=subtract)

def divide(a: float, b: float) -> float:
    """Divide two numbers and returns the quotient"""
    return a / b

divide_tool = FunctionTool.from_defaults(fn=divide)

In [53]:
from typing import Generator
from io import StringIO
import sys
import threading
import queue
import time
import re
from gradio import ChatMessage

def get_math_response(question: str, history) -> Generator[list[ChatMessage], None, None]:
    output_queue = queue.Queue()
    
    class QueuedStringIO(StringIO):
        def write(self, txt):
            if txt.strip():
                output_queue.put(txt)
            return super().write(txt)
    
    string_buffer = QueuedStringIO()
    old_stdout = sys.stdout
    sys.stdout = string_buffer
    
    try:
        def run_agent():
            llm = OpenAI(model="gpt-4", temperature=0)
            agent = ReActAgent.from_tools(
                [multiply_tool, add_tool, subtract_tool, divide_tool], 
                llm=llm, 
                verbose=True
            )
            response = agent.chat(question)
            output_queue.put(f"Final answer: {str(response)}")
            output_queue.put(None)
        
        thread = threading.Thread(target=run_agent)
        thread.start()
        
        messages = []
        # Regex patterns
        thought_pattern = r"Thought: (.*?)(?=Action:|Answer:|$)"
        action_pattern = r"Action: (\w+)\nAction Input: ({.*?})"
        observation_pattern = r"Observation: (.*?)(?=\[|$)"
        answer_pattern = r"Answer: (.*?)(?=\[|$)"
        
        while True:
            try:
                line = output_queue.get(timeout=0.1)
                if line is None:
                    break
                    
                # Match thought
                thought_match = re.search(thought_pattern, line, re.DOTALL)
                if thought_match:
                    thought = thought_match.group(1).strip()
                    messages.append(ChatMessage(
                        metadata={"title": "Thought"},
                        content=thought
                    ))
                    yield messages
                
                # Match action and observation
                action_match = re.search(action_pattern, line, re.DOTALL)
                obs_match = re.search(observation_pattern, line)
                if action_match and obs_match:
                    action = action_match.group(1)
                    action_input = action_match.group(2)
                    observation = obs_match.group(1).strip()
                    messages.append(ChatMessage(
                        metadata={"title": f"Action: {action} {action_input}"},
                        content=f"Observation: {observation}"
                    ))
                    yield messages
                
                # Match answer
                answer_match = re.search(answer_pattern, line)
                if answer_match:
                    answer = answer_match.group(1).strip()
                    messages.append(ChatMessage(
                        content=f"Answer: {answer}"
                    ))
                    yield messages
                
            except queue.Empty:
                continue
            
    finally:
        sys.stdout = old_stdout
        string_buffer.close()

In [54]:
import gradio as gr

gr.ChatInterface(get_math_response, type="messages", chatbot=gr.Chatbot(type="messages")).launch(inline=False)


* Running on local URL:  http://127.0.0.1:7878

To create a public link, set `share=True` in `launch()`.


In [30]:
i

'Final answer: The result of the expression 20+(2*4) is 28.'